## Import Libraries

In [93]:
import requests  # for HTTPS requests to URLS
import zipfile  # to un-zip any data files
import pypdf  # to extract text from PDFs (is NOT that good)
import sys, pathlib, fitz  # to extract text from PDFs
import bs4  # HTLM parser to extract data from HTML pages
import math
from sympy import divisors

In [2]:
BASE_GOVINFO = "https://api.govinfo.gov/"

In [3]:
r = requests.get(url="https://www.govinfo.gov/bulkdata/SCD/1937/SCD-1937.zip")

In [4]:
open("data/SCD-1937.zip", "wb").write(r.content)

52392953

In [7]:
with zipfile.ZipFile(file="data/SCD-1937.zip", mode="r") as zip_ref:
    zip_ref.extractall("data")

In [8]:
# Loop for 1991 to 2009
for volume in range(502,561):
    url = f"https://www.supremecourt.gov/opinions/boundvolumes/{volume}bv.pdf"
    r = requests.get(url=url)
    open(f"data/SCD-{volume}.pdf", "wb").write(r.content)
    break

# Loop for 2009 to 2015
for volume in range(561,578):
    url = f"https://www.supremecourt.gov/opinions/boundvolumes/{volume}BV.pdf"

In [3]:
reader = pypdf.PdfReader("data/SCD-502.pdf")
reader.pages[0].extract_text()

'502BV$spin 08-02-95 10:19:40\nUNITED STATES\nREPORTS\n502\nOCT. TERM 1991'

In [4]:
with fitz.open("data/SCD-502.pdf") as doc:  # open document
    text = chr(12).join([page.get_text() for page in doc])
# write as a binary file to support non-ASCII characters
pathlib.Path("data/SCD-502" + ".txt").write_bytes(text.encode())


2421174

421

In [ ]:
# Get counts for page requests
url = "https://www.presidency.ucsb.edu/documents/app-categories/presidential/memoranda?items_per_page=5"
r = requests.get(url=url)
soup = bs4.BeautifulSoup(markup=r.content, features="html.parser")
count = soup.find(attrs={"class":"tax-count"}).text
total = int(count[count.index("of")+3: count.index(".")])
MAX_DISPLAY = 500
display = max(map(lambda x: x if x<MAX_DISPLAY else 0,divisors(total)))

# Loop to gather all memordandas
for page in range(total/display):
    url = f"https://www.presidency.ucsb.edu/documents/app-attributes/determinations?items_per_page={display}&page={page}"
    r = requests.get(url=url)
    soup = bs4.BeautifulSoup(markup=r.content, features="html.parser")
    memorandas = [child.a["href"] for child in soup.find_all(attrs={"class":"field-title"})]

    for href in memorandas:
        # get content
        r = requests.get(url=f"https://www.presidency.ucsb.edu{href}")
        soup = bs4.BeautifulSoup(markup=r.content, features="html.parser")
        content = ""
        for child in soup.find(attrs={"class":"field-docs-content"}).children:
            content += child.text
        # Get person
        soup = bs4.BeautifulSoup(markup=r.content, features="html.parser")
        person = ""
        for child in soup.find(attrs={"class":"diet-title"}).children:
            person += child.text



In [65]:
memorandas = [child.a["href"] for child in soup.find_all(attrs={"class":"field-title"})]
    

In [69]:
memorandas[0]

'/documents/memorandum-delegation-authority-under-section-506a1-the-foreign-assistance-act-1961-32'

In [76]:
for href in memorandas:
    # get content
    r = requests.get(url=f"https://www.presidency.ucsb.edu{href}")
    soup = bs4.BeautifulSoup(markup=r.content, features="html.parser")
    content = ""
    for child in soup.find(attrs={"class":"field-docs-content"}).children:
        content += child.text
    # Get person
    soup = bs4.BeautifulSoup(markup=r.content, features="html.parser")
    person = ""
    for child in soup.find(attrs={"class":"diet-title"}).children:
        person += child.text


    break
    


In [78]:
person

'Joseph R. Biden'

In [ ]:
"https://www.presidency.ucsb.edu/documents/app-attributes/determinations?items_per_page=60"
"https://www.presidency.ucsb.edu/documents/app-attributes/determinations?items_per_page=60&page=1"

In [ ]:
"https://www.presidency.ucsb.edu/documents/app-attributes/executive-orders?items_per_page=60"
"https://www.presidency.ucsb.edu/documents/app-attributes/executive-orders?items_per_page=60&page=1"

In [ ]:
"https://www.presidency.ucsb.edu/documents/app-categories/written-presidential-orders/presidential/proclamations?items_per_page=60"
"https://www.presidency.ucsb.edu/documents/app-categories/written-presidential-orders/presidential/proclamations?items_per_page=60&page=1"